<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models 
from tensorflow import keras

<p>To understand how resnet work read </p>
<a href='https://arxiv.org/pdf/1512.03385.pdf'>Deep Residual Learning for Image Recognition<a/>
    
<a href='https://arxiv.org/pdf/1603.05027.pdf'>Identity Mappings in Deep Residual Networks</a>
<img src="images/resnet.jpg"  width='1000px'>
Source:  <a href='https://arxiv.org/pdf/1512.03385.pdf'>Deep Residual Learning for Image Recognition<a/>
  

In [2]:
class Residual(keras.models.Model):
    def __init__(self,num_filters,stride=1,downsample=False,**kwargs):
        super().__init__(**kwargs)
        self.cov1=layers.Conv2D(num_filters,kernel_size=3,strides=stride,padding='same')
        self.cov2=layers.Conv2D(num_filters,kernel_size=3,padding='same')
        if downsample:
            self.downsample=layers.Conv2D(num_filters,kernel_size=1,strides=stride)
        else:
            self.downsample=None
        self.bn1=layers.BatchNormalization()
        self.bn2=layers.BatchNormalization()
        self.relu=layers.ReLU()
    def call(self,x):
        hx=self.relu(self.bn1(self.cov1(x)))
        hx=self.bn2(self.cov2(hx))
        if self.downsample:
            x=self.downsample(x)
        return self.relu(x+hx)

In [3]:
def resnet_block(num_filters,num_residuals,first_block=False):
    blk=models.Sequential()
    for residual in range(num_residuals):
        if residual==0 and not first_block:
            blk.add(Residual(num_filters=num_filters,downsample=True,stride=2))
        else:
            blk.add(Residual(num_filters=num_filters))
    return blk
            

In [4]:
resnet34=models.Sequential([
          layers.Conv2D(64,kernel_size=7,strides=2,padding='same'),
          layers.BatchNormalization(),
          layers.Activation('relu'),
          layers.MaxPool2D(pool_size=3,strides=2,padding='same'),
          resnet_block(64,3,first_block=True),
          resnet_block(128,4),
          resnet_block(256,6),
          resnet_block(512,3),
          layers.GlobalAvgPool2D(),
          layers.Flatten(),
          layers.Dense(10)
         ])

In [5]:
X = tf.random.uniform(shape=(1, 224, 224,1))
for layer in resnet34.layers:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Conv2D output shape:	 (1, 112, 112, 64)
BatchNormalization output shape:	 (1, 112, 112, 64)
Activation output shape:	 (1, 112, 112, 64)
MaxPooling2D output shape:	 (1, 56, 56, 64)
Sequential output shape:	 (1, 56, 56, 64)
Sequential output shape:	 (1, 28, 28, 128)
Sequential output shape:	 (1, 14, 14, 256)
Sequential output shape:	 (1, 7, 7, 512)
GlobalAveragePooling2D output shape:	 (1, 512)
Flatten output shape:	 (1, 512)
Dense output shape:	 (1, 10)
